In [547]:
#### import global modules
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from yaml import safe_load
import google.oauth2.credentials
from google.cloud import bigquery
import gc

# Set global vars
pth_project = Path(os.getcwd().split('notebooks')[0])
pth_data = pth_project / 'data'
pth_queries = pth_project / 'core' / 'queries'
pth_creds = pth_project / 'conf' / 'local' / 'project_config.yaml'
sys.path.insert(0, str(pth_project))
d_project_config = safe_load(pth_creds.open())
# d_params = safe_load((pth_project / 'core' / 'parameters' / 'common.yaml').open())['data_extract']

# import local modules
from core.utils.gcp import connect_bq_services
# from core.etl.extract import extract_bq_data, extract_pr_codes, format_conv_df, filter_convs

# Connect to google services
bq_client = connect_bq_services(d_project_config['gcp-project-name'])
pd.options.display.max_rows = 100

In [548]:
def extract_bq_data(bq_client, sql=None, pth_query=None):
    if sql is not None:
        df = bq_client.query(sql).to_dataframe()
    elif pth_query is not None:
        sql = pth_query.read_text()
        df = bq_client.query(sql).to_dataframe()
    else:
        raise ValueError('`sql` or `pth_query` should be set')  
    return df

In [549]:
Telus_Customer_details='''



with ADC_customer_base as(

SELECT distinct customer_id,dealer_customer_id,dealer_name,account_type_name,customer_type_name
FROM `cio-datahub-enterprise-pr-183a.src_adc.bq_customer_account_details` 
where date(last_updt_ts) = '2022-10-31'
--and account_type_name!='Standalone'
--and customer_type_name='Customer'
and dealer_name='TELUS Communications Inc.'
QUALIFY ROW_NUMBER() OVER (PARTITION BY dealer_customer_id ORDER BY last_updt_ts DESC) = 1
--order by dealer_name,join_date,customer_id
)
,

Telus_customers as
(
select bacct_bus_bacct_num,bacct_billg_acct_id,cust_bus_cust_id,pi_prod_instnc_typ_cd,pi_prod_instnc_stat_ts,prod_instnc_ts,pi_cntrct_end_ts as contract_end_date
from `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` 
WHERE DATE(prod_instnc_ts) = "2022-10-31" #Snapshot of the last day of the month
and pi_prod_instnc_typ_cd ='SMHM' #Serice type
and bus_prod_instnc_src_id = 1001 #BANs that are for home services
and pi_prod_instnc_stat_cd in ('A')
and  consldt_cust_typ_cd = 'R'
QUALIFY ROW_NUMBER() OVER (PARTITION BY cust_bus_cust_id ORDER BY prod_instnc_ts DESC) = 1
order by cust_bus_cust_id

)


,Telus_internet_customers as 
(

select distinct (customer_id) as cust_id_telus,Telus_internet_flag from `divgpras-pr-579355.ADC_updated.Telus_internet_31Oct2022`

)



select * from ADC_customer_base as ADC
inner join Telus_customers as Telus
on ADC.dealer_customer_id=Telus.cust_bus_cust_id
left join Telus_internet_customers
on ADC.customer_id=Telus_internet_customers.cust_id_telus

'''

In [550]:
Data_Oct2022=extract_bq_data(bq_client, sql=Telus_Customer_details)

In [552]:
Data_Oct2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 326670 entries, 0 to 326669
Data columns (total 14 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   customer_id             326670 non-null  Int64 
 1   dealer_customer_id      326670 non-null  object
 2   dealer_name             326670 non-null  object
 3   account_type_name       326670 non-null  object
 4   customer_type_name      326670 non-null  object
 5   bacct_bus_bacct_num     326670 non-null  Int64 
 6   bacct_billg_acct_id     326670 non-null  Int64 
 7   cust_bus_cust_id        326670 non-null  object
 8   pi_prod_instnc_typ_cd   326670 non-null  object
 9   pi_prod_instnc_stat_ts  326670 non-null  object
 10  prod_instnc_ts          326670 non-null  object
 11  contract_end_date       326670 non-null  object
 12  cust_id_telus           230927 non-null  Int64 
 13  Telus_internet_flag     230927 non-null  Int64 
dtypes: Int64(5), object(9)
memory usage:

In [553]:
# Data_Oct2022['Telus_Churn_Flag'].sum()

In [554]:
Data_Oct2022['Telus_internet_flag'].sum()

230927

In [555]:
# Data_Oct2022['Telus_Churn_Flag'].fillna(0,inplace=True)

In [556]:
Data_Oct2022['Telus_internet_flag'].fillna(0,inplace=True)

In [557]:
Data_Oct2022['Telus_internet_flag'].value_counts(normalize=True)*100

1    70.691217
0    29.308783
Name: Telus_internet_flag, dtype: Float64

In [558]:
Data_Oct2022['dealer_customer_id'].isna().sum()

0

In [559]:
Data_Oct2022['customer_id'].value_counts()

13900254    1
13878651    1
10369728    1
13670506    1
11935249    1
           ..
12090667    1
12324607    1
13124901    1
11833060    1
12864950    1
Name: customer_id, Length: 326670, dtype: Int64

In [560]:
Data_Oct2022['dealer_customer_id'].value_counts()

100914267    1
20790639     1
2612908      1
2591957      1
2567689      1
            ..
18872049     1
18828218     1
1870974      1
1865754      1
99996141     1
Name: dealer_customer_id, Length: 326670, dtype: int64

In [561]:
Data_Oct2022['bacct_bus_bacct_num'].isna().sum()*100/Data_Oct2022.shape[0]

0.0

In [562]:
pd.DataFrame(Data_Oct2022.groupby(['dealer_name']).agg(
    
    Customer_count= ('customer_id','nunique'),
    Churn_internet_total=('Telus_internet_flag',lambda x: x.sum()),
    Churn_internet_rate=('Telus_internet_flag',lambda x: x.mean()*100))
             .reset_index())

,dealer_name,Customer_count,Churn_internet_total,Churn_internet_rate
0,TELUS Communications Inc.,326670,230927,70.691217


In [563]:
pd.DataFrame(Data_Oct2022.groupby(['account_type_name']).agg(
    
    Customer_count= ('customer_id','nunique'),
    Churn_internet_total=('Telus_internet_flag',lambda x: x.sum()),
    Churn_internet_rate=('Telus_internet_flag',lambda x: x.mean()*100))
             .reset_index())

,account_type_name,Customer_count,Churn_internet_total,Churn_internet_rate
0,Awareness and Automation,82611,73563,89.047463
1,Security System,199060,124935,62.762484
2,Standalone,44999,32429,72.066046


In [564]:
pd.DataFrame(Data_Oct2022.groupby(['customer_type_name']).agg(
    
    Customer_count= ('customer_id','nunique'),
    Churn_internet_total=('Telus_internet_flag',lambda x: x.sum()),
    Churn_internet_rate=('Telus_internet_flag',lambda x: x.mean()*100))
             .reset_index())

,customer_type_name,Customer_count,Churn_internet_total,Churn_internet_rate
0,Commitment,4777,3473,72.702533
1,Customer,321791,227379,70.660460
2,Demo - Normal,99,75,75.757576
3,From Recycle,3,0,0.000000


In [565]:
pd.DataFrame(Data_Oct2022.groupby(['customer_type_name','account_type_name']).agg(
    
    Customer_count= ('customer_id','nunique'),
    Telus_matched_customers=('cust_bus_cust_id','nunique'),
    Telus_internet_customers=('Telus_internet_flag',lambda x: x.sum()),
    Telus_internet_penetration=('Telus_internet_flag',lambda x: x.mean()*100))
             .reset_index())

,customer_type_name,account_type_name,Customer_count,Telus_matched_customers,Telus_internet_customers,Telus_internet_penetration
0,Commitment,Awareness and Automation,3284,3284,2763,84.135201
1,Commitment,Security System,1484,1484,702,47.304582
2,Commitment,Standalone,9,9,8,88.888889
3,Customer,Awareness and Automation,79327,79327,70800,89.250823
4,Customer,Security System,197474,197474,124158,62.873087
5,Customer,Standalone,44990,44990,32421,72.062681
6,Demo - Normal,Security System,99,99,75,75.757576
7,From Recycle,Security System,3,3,0,0.000000


In [566]:
# Data_Oct2022[Data_Oct2022['account_type_name']=='Standalone'][['customer_id','cust_bus_cust_id','account_type_name']]

In [567]:
Best_Practices_Query= '''

select * from `divgpras-pr-579355.ADC_updated.Best_Practices_Customers_latest`

'''

In [568]:
Best_Practices_DF=extract_bq_data(bq_client, sql=Best_Practices_Query)

In [569]:
Best_Practices_DF.head()

,customer_id,Best_partices_1,Best_partices_2,Best_partices_3,Best_partices_4,Best_partices_5,Best_partices_6,Best_Practice_All,Best_Practice_All_flag
0,11254267,1,1,0,1,1,0,4,0
1,10445690,1,1,0,1,1,0,4,0
2,6579995,1,1,0,1,1,0,4,0
3,11253105,1,1,0,1,1,0,4,0
4,11253845,1,1,0,1,1,0,4,0


In [570]:
Arming_Query=  '''

with date_sql as (
select  count(distinct date(date)) as count_of_dates from `cio-datahub-enterprise-pr-183a.src_adc.bq_aggregate_daily_arming_commands`
where date(date)< '2022-11-01' and date(date)>= '2022-10-01'

)

select id_cust as customer_id,sum(count_arm_commands) as sum_arm_commands,sum(count_disarm_commands) as sum_disarm_commands,count (distinct date(date)) as number_days_arming_disarming, (select count_of_dates from date_sql ) as count_of_dates_arming
from `cio-datahub-enterprise-pr-183a.src_adc.bq_aggregate_daily_arming_commands`
where date(date)< '2022-11-01' and date(date)>= '2022-10-01'
group by customer_id
order by number_days_arming_disarming desc

'''

In [571]:
Arming_DF=extract_bq_data(bq_client, sql=Arming_Query)

In [572]:
Arming_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 326507 entries, 0 to 326506
Data columns (total 5 columns):
 #   Column                        Non-Null Count   Dtype
---  ------                        --------------   -----
 0   customer_id                   326507 non-null  Int64
 1   sum_arm_commands              326507 non-null  Int64
 2   sum_disarm_commands           326507 non-null  Int64
 3   number_days_arming_disarming  326507 non-null  Int64
 4   count_of_dates_arming         326507 non-null  Int64
dtypes: Int64(5)
memory usage: 14.0 MB


In [573]:

max_number_arming_dates=Arming_DF['count_of_dates_arming'].max()
Arming_DF['Arming_Consistency']=Arming_DF['number_days_arming_disarming'].apply(lambda x: x*100/max_number_arming_dates)


In [574]:
max_number_arming_dates

30

In [575]:
Arming_DF['Arming_Flag']=1

In [576]:
Login_SQL='''

with date_sql as (
select  count(distinct date(login_dt_utc)) as count_of_dates from `cio-datahub-enterprise-pr-183a.src_adc.bq_customer_daily_logins`
where date(login_dt_utc)< '2022-11-01' and date(login_dt_utc)>= '2022-10-01' 

)



select customer_id,sum(login_count) as sum_login_count,count (distinct date(login_dt_utc)) as number_of_login_days, (select count_of_dates from date_sql ) as count_of_dates_logins
from `cio-datahub-enterprise-pr-183a.src_adc.bq_customer_daily_logins`
where date(login_dt_utc)< '2022-11-01' and date(login_dt_utc)>= '2022-10-01' 
group by customer_id
order by number_of_login_days desc


'''

In [577]:
Login_DF=extract_bq_data(bq_client, sql=Login_SQL)

In [578]:
max_number_login_dates=Login_DF['count_of_dates_logins'].max()
Login_DF['Login_Consistency']=Login_DF['number_of_login_days'].apply(lambda x: x*100/max_number_login_dates)

In [579]:
max_number_login_dates

15

In [580]:
Login_DF.head()

,customer_id,sum_login_count,number_of_login_days,count_of_dates_logins,Login_Consistency
0,6971873,118,15,15,100.0
1,9455216,67,15,15,100.0
2,8265537,281,15,15,100.0
3,11002802,75,15,15,100.0
4,6988451,139,15,15,100.0


In [581]:
Camera_TC_issue_query='''

select dealer_name,customer_id,unit_id,device_id,trouble_condition_name,trouble_condition_group_id,trouble_condition_group_desc, start_date_utc,end_date_utc
from `cio-datahub-enterprise-pr-183a.src_adc.bq_troublecondition_data`
where trouble_condition_start_ind='Y'and trouble_condition_closed_ind='Y' 
and date(start_date_utc) >= '2022-08-01' and date(start_date_utc) <'2022-11-01' 
and trouble_condition_name in ('CameraNotReachable','CameraNotCommunicating','BroadbandCommFailure')


'''

In [582]:
Camera_TC= extract_bq_data(bq_client, sql=Camera_TC_issue_query)

In [583]:
Camera_TC['count_instance']=1

TC_pivot_table=pd.pivot_table(Camera_TC, values=['count_instance'], index=['customer_id'], columns='trouble_condition_name',
                          aggfunc={'count_instance': np.sum})

In [584]:
TC_pivot_table=TC_pivot_table.reset_index()

In [585]:
TC_pivot_table.head()

customer_id       count_instance  \
trouble_condition_name             BroadbandCommFailure   
0                           213860                 44.0   
1                           218873                  NaN   
2                           229153                  NaN   
3                           233626                  5.0   
4                           287850                 10.0   

                                                                  
trouble_condition_name CameraNotCommunicating CameraNotReachable  
0                                         NaN                NaN  
1                                         NaN                2.0  
2                                         NaN                1.0  
3                                         NaN                NaN  
4                                         NaN                NaN

In [586]:
TC_pivot_table.columns=['customer_id','BroadbandCommFailure_count','CameraNotCommunicating_count','CameraNotReachable_count']

In [587]:
TC_pivot_table.fillna(0,inplace=True)

In [588]:
TC_pivot_table['BroadbandCommFailure_Flag']=TC_pivot_table['BroadbandCommFailure_count'].apply(lambda x: 1 if x>0 else 0)
TC_pivot_table['CameraNotCommunicating_Flag']=TC_pivot_table['CameraNotCommunicating_count'].apply(lambda x: 1 if x>0 else 0)
TC_pivot_table['CameraNotReachable_Flag']=TC_pivot_table['CameraNotReachable_count'].apply(lambda x: 1 if x>0 else 0)


In [589]:
Merge_DF=Data_Oct2022.merge(Best_Practices_DF,on='customer_id',how='left')

In [590]:
Merge_DF_1=Merge_DF.merge(Arming_DF,on='customer_id',how='left')

In [591]:
Merge_DF_2=Merge_DF_1.merge(Login_DF,on='customer_id',how='left')

In [592]:
Merge_DF_3=Merge_DF_2.merge(TC_pivot_table,on='customer_id',how='left')

In [593]:
Merge_DF_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 326670 entries, 0 to 326669
Data columns (total 38 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   customer_id                   326670 non-null  Int64  
 1   dealer_customer_id            326670 non-null  object 
 2   dealer_name                   326670 non-null  object 
 3   account_type_name             326670 non-null  object 
 4   customer_type_name            326670 non-null  object 
 5   bacct_bus_bacct_num           326670 non-null  Int64  
 6   bacct_billg_acct_id           326670 non-null  Int64  
 7   cust_bus_cust_id              326670 non-null  object 
 8   pi_prod_instnc_typ_cd         326670 non-null  object 
 9   pi_prod_instnc_stat_ts        326670 non-null  object 
 10  prod_instnc_ts                326670 non-null  object 
 11  contract_end_date             326670 non-null  object 
 12  cust_id_telus                 230927 non-nul

In [594]:
Merge_DF_3['Best_Practice_All_flag'].isna().sum()

41529

In [595]:
Merge_DF_3['Arming_Consistency'].isna().sum()

207686

In [596]:
Merge_DF_3['Login_Consistency'].isna().sum()

172684

In [597]:
Merge_DF_3['Login_Consistency'].value_counts()

6.666667     153976
40.000000         3
33.333333         3
20.000000         2
66.666667         1
60.000000         1
Name: Login_Consistency, dtype: int64

In [598]:
Merge_DF_3.fillna(0,inplace=True)

In [599]:
def Segment_making(row
                   ,min_login_consistency_threshold=50
                   ,min_arming_consistency_threshold=50
                   ,max_login_consistency_threshold=50
                   ,max_arming_consistency_threshold=50):
    
    if row['Best_Practice_All']>=5 and row['Login_Consistency']>=max_login_consistency_threshold  and row['Arming_Consistency']>=max_arming_consistency_threshold:
        return 'Heavy_User'
    elif row['Login_Consistency']>=max_login_consistency_threshold  and row['Arming_Consistency']< min_arming_consistency_threshold:
        return 'Home_automation_Savvy'
    elif row['Login_Consistency']< min_login_consistency_threshold  and row['Arming_Consistency']>=max_login_consistency_threshold:
        return 'Old_Fashion'
    elif row['Login_Consistency']==0  and row['Arming_Consistency']==0:
        return 'Disengaged'
    else:
        return "Moderate_Users"

In [600]:
Merge_DF_3['Segment']=Merge_DF_3.apply(Segment_making,axis=1)

In [601]:
pd.DataFrame(Merge_DF_3[Merge_DF_3['Best_partices_1']==0].groupby(['Segment']).agg(
    
    Customer_count= ('customer_id','nunique'),
    Best_practices_1=('Best_partices_1','sum'),
     Login_conis=('Login_Consistency','mean'),
    # Telus_internet_customers=('Telus_internet_flag',lambda x: x.sum()),
    # Telus_internet_penetration=('Telus_internet_flag',lambda x: x.mean()*100),
    # Churn_total=('Telus_Churn_Flag',lambda x: x.sum()),
    # Churn_rate=('Telus_Churn_Flag',lambda x: x.mean()*100)
).reset_index())

,Segment,Customer_count,Best_practices_1,Login_conis
0,Disengaged,42638,0,0.000000
1,Moderate_Users,23182,0,5.267880
2,Old_Fashion,8304,0,4.418754


In [602]:
pd.DataFrame(Merge_DF_3.groupby(['Segment']).agg(
    
    Customer_count= ('customer_id','nunique'),
    Best_practices_1=('Best_partices_1','sum'),
     Login_conis=('Login_Consistency','mean'),
    # Telus_internet_customers=('Telus_internet_flag',lambda x: x.sum()),
    # Telus_internet_penetration=('Telus_internet_flag',lambda x: x.mean()*100),
    # Churn_total=('Telus_Churn_Flag',lambda x: x.sum()),
    # Churn_rate=('Telus_Churn_Flag',lambda x: x.mean()*100)
).reset_index())

,Segment,Customer_count,Best_practices_1,Login_conis
0,Disengaged,132136,89498,0.000000
1,Moderate_Users,125722,102540,5.537031
2,Old_Fashion,68812,60508,4.806817


In [603]:
Merge_DF_3['Segment'].value_counts(normalize=True)*100

Disengaged        40.449383
Moderate_Users    38.485934
Old_Fashion       21.064683
Name: Segment, dtype: float64

In [604]:
Merge_DF_3['Segment'].value_counts()

Disengaged        132136
Moderate_Users    125722
Old_Fashion        68812
Name: Segment, dtype: int64

In [605]:
Merge_DF_3['Login_Consistency'].value_counts(normalize=True)*100

0.000000     52.861910
6.666667     47.135029
40.000000     0.000918
33.333333     0.000918
20.000000     0.000612
66.666667     0.000306
60.000000     0.000306
Name: Login_Consistency, dtype: float64

In [606]:
Merge_DF_3['Login_Consistency'].value_counts()

0.000000     172684
6.666667     153976
40.000000         3
33.333333         3
20.000000         2
66.666667         1
60.000000         1
Name: Login_Consistency, dtype: int64

In [607]:
Merge_DF_3['Arming_Consistency'].value_counts(normalize=True)*100

0.000000      63.576698
100.000000     7.110540
3.333333       4.282915
96.666667      2.786910
6.666667       2.629871
93.333333      1.878042
10.000000      1.441822
90.000000      1.380598
86.666667      1.142743
13.333333      1.117336
83.333333      0.919582
16.666667      0.882236
80.000000      0.830196
20.000000      0.754890
76.666667      0.703156
23.333333      0.673769
73.333333      0.666422
70.000000      0.601524
26.666667      0.586525
66.666667      0.580402
63.333333      0.552852
33.333333      0.535097
30.000000      0.534484
56.666667      0.500505
60.000000      0.495607
40.000000      0.488566
43.333333      0.487648
36.666667      0.482138
53.333333      0.464689
46.666667      0.460710
50.000000      0.451526
Name: Arming_Consistency, dtype: float64

In [608]:
Merge_DF_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 326670 entries, 0 to 326669
Data columns (total 39 columns):
 #   Column                        Non-Null Count   Dtype              
---  ------                        --------------   -----              
 0   customer_id                   326670 non-null  Int64              
 1   dealer_customer_id            326670 non-null  object             
 2   dealer_name                   326670 non-null  object             
 3   account_type_name             326670 non-null  object             
 4   customer_type_name            326670 non-null  object             
 5   bacct_bus_bacct_num           326670 non-null  Int64              
 6   bacct_billg_acct_id           326670 non-null  Int64              
 7   cust_bus_cust_id              326670 non-null  object             
 8   pi_prod_instnc_typ_cd         326670 non-null  object             
 9   pi_prod_instnc_stat_ts        326670 non-null  datetime64[ns]     
 10  prod_instnc_ts      

In [609]:
Merge_DF_3['contract_end_date'].value_counts()

9999-12-31 00:00:00    30788
2025-10-26 00:00:00      584
2025-10-27 00:00:00      577
2025-10-20 00:00:00      570
2025-10-21 00:00:00      558
                       ...  
2019-10-23 00:00:00        1
2013-02-05 00:00:00        1
2008-11-22 00:00:00        1
2017-06-18 00:00:00        1
1205-09-11 00:00:00        1
Name: contract_end_date, Length: 4226, dtype: int64

In [610]:
# Merge_DF_2[Merge_DF_2['contract_end_date']]

In [611]:
Merge_DF_3['Camera_TC_condition_count']=Merge_DF_3.apply(lambda x: x['BroadbandCommFailure_count']+x['CameraNotCommunicating_count']+x['CameraNotReachable_count'],axis=1)

In [612]:
Merge_DF_3['Camera_TC_condition_count'].value_counts()

0.0       272538
1.0        23731
2.0         9231
3.0         5021
4.0         3809
           ...  
301.0          1
486.0          1
1271.0         1
496.0          1
1022.0         1
Name: Camera_TC_condition_count, Length: 638, dtype: int64

In [613]:
Merge_DF_3['Best_partices_5'].value_counts(normalize=True)*100

1    66.601463
0    33.398537
Name: Best_partices_5, dtype: Float64

In [617]:
Merge_DF_3.shape

(326670, 40)

In [618]:
Merge_DF_3_sub=Merge_DF_3[Merge_DF_3['account_type_name']=='Security System']

In [619]:
Merge_DF_3_sub.shape

(199060, 40)

In [621]:
Merge_DF_3_sub.to_csv('GTM_full_data_Nov2022.csv')

In [622]:
Merge_DF_3_sub.to_csv('GTM_full_data_Nov2022.txt',sep=' ', index=False)

In [ ]:
# Merge_DF_3_sub.to_

In [614]:
# Merge_DF_3['Telus_Churn_Flag'].value_counts(normalize=True)*100

In [536]:
GTM_data_8=Merge_DF_3[(Merge_DF_3['dealer_name']=='TELUS Communications Inc.') & ((Merge_DF_3['Segment']=='Disengaged') | (Merge_DF_3['Segment']=='Moderate_Users'))]

In [537]:
GTM_data_8.shape

(257847, 40)

In [539]:
GTM_data_8['Best_partices_3'].value_counts()

0    180674
1     77173
Name: Best_partices_3, dtype: Int64

In [543]:
pd.DataFrame(Merge_DF_3[Merge_DF_3['account_type_name']=='Security System'].groupby(['Segment']).agg(
    
    Customer_count= ('customer_id','nunique'),
    Best_practices_3=('Best_partices_3','sum'),
     # Login_conis=('Login_Consistency','mean'),
    # Telus_internet_customers=('Telus_internet_flag',lambda x: x.sum()),
    # Telus_internet_penetration=('Telus_internet_flag',lambda x: x.mean()*100),
    # Churn_total=('Telus_Churn_Flag',lambda x: x.sum()),
    # Churn_rate=('Telus_Churn_Flag',lambda x: x.mean()*100)
).reset_index())

,Segment,Customer_count,Best_practices_3
0,Disengaged,43844,26880
1,Moderate_Users,86420,50284
2,Old_Fashion,68787,42110


In [545]:
Merge_DF_3.columns.to_list()

['customer_id',
 'dealer_customer_id',
 'dealer_name',
 'account_type_name',
 'customer_type_name',
 'bacct_bus_bacct_num',
 'bacct_billg_acct_id',
 'cust_bus_cust_id',
 'pi_prod_instnc_typ_cd',
 'pi_prod_instnc_stat_ts',
 'prod_instnc_ts',
 'contract_end_date',
 'cust_id_telus',
 'Telus_internet_flag',
 'Best_partices_1',
 'Best_partices_2',
 'Best_partices_3',
 'Best_partices_4',
 'Best_partices_5',
 'Best_partices_6',
 'Best_Practice_All',
 'Best_Practice_All_flag',
 'sum_arm_commands',
 'sum_disarm_commands',
 'number_days_arming_disarming',
 'count_of_dates_arming',
 'Arming_Consistency',
 'Arming_Flag',
 'sum_login_count',
 'number_of_login_days',
 'count_of_dates_logins',
 'Login_Consistency',
 'BroadbandCommFailure_count',
 'CameraNotCommunicating_count',
 'CameraNotReachable_count',
 'BroadbandCommFailure_Flag',
 'CameraNotCommunicating_Flag',
 'CameraNotReachable_Flag',
 'Segment',
 'Camera_TC_condition_count']

In [467]:
GTM_data_7=Merge_DF_3[(Merge_DF_3['dealer_name']=='TELUS Communications Inc.')& (Merge_DF_3['Best_partices_5']==1)  & ((Merge_DF_3['Segment']=='Disengaged') | (Merge_DF_3['Segment']=='Moderate_Users'))]

In [468]:
GTM_data_7.shape

(172441, 40)

In [469]:
GTM_data_7.to_csv('GTM_data_HomeAutodev_Nov.csv',index=False)

In [396]:
GTM_dis_data_1=Merge_DF_3[(Merge_DF_3['dealer_name']=='TELUS Communications Inc.') & (Merge_DF_3['Best_partices_1']==0) & ((Merge_DF_3['Segment']=='Disengaged') | (Merge_DF_3['Segment']=='Moderate_Users'))] 
GTM_dis_data_2=Merge_DF_3[(Merge_DF_3['dealer_name']=='TELUS Communications Inc.') & (Merge_DF_3['Best_Practice_All']<3) & ((Merge_DF_3['Segment']=='Disengaged') | (Merge_DF_3['Segment']=='Moderate_Users'))] 
GTM_dis_data_3=Merge_DF_3[(Merge_DF_3['dealer_name']=='TELUS Communications Inc.') & (Merge_DF_3['Camera_TC_condition_count']>3) & ((Merge_DF_3['Segment']=='Disengaged') | (Merge_DF_3['Segment']=='Moderate_Users'))] 


In [325]:
GTM_dis_data_1['Telus_internet_flag'].value_counts(normalize=True)*100

1    69.42405
0    30.57595
Name: Telus_internet_flag, dtype: Float64

In [326]:
GTM_dis_data_2['Telus_internet_flag'].value_counts(normalize=True)*100

1    69.108379
0    30.891621
Name: Telus_internet_flag, dtype: Float64

In [327]:
GTM_dis_data_3['Telus_internet_flag'].value_counts(normalize=True)*100

1    69.997989
0    30.002011
Name: Telus_internet_flag, dtype: Float64

In [328]:
GTM_dis_data_1.to_csv('GTM_data_Nov2022_NoEV.csv',index=False)

In [329]:
GTM_dis_data_2.to_csv('GTM_data_Nov2022_BPlessthan3.csv',index=False)

In [330]:
GTM_dis_data_3.to_csv('GTM_data_Nov2022_CameraTC_grtr_than3.csv',index=False)

In [82]:
# GTM_Mod_data_1=Merge_DF_3[(Merge_DF_3['dealer_name']=='TELUS Communications Inc.') & (Merge_DF_3['Best_partices_1']==0) & (Merge_DF_3['Segment']=='Moderate_Users')] 
# GTM_Mod_data_2=Merge_DF_3[(Merge_DF_3['dealer_name']=='TELUS Communications Inc.') & (Merge_DF_3['Best_Practice_All']<3) & (Merge_DF_3['Segment']=='Moderate_Users')] 
# GTM_Mod_data_3=Merge_DF_3[(Merge_DF_3['dealer_name']=='TELUS Communications Inc.') & (Merge_DF_3['Camera_TC_condition_count']>3) & (Merge_DF_3['Segment']=='Moderate_Users')]

In [83]:
# GTM_Mod_data_1.to_csv('GTM_data_NoEV_Moderate.csv',index=False)
# GTM_Mod_data_2.to_csv('GTM_data_BPlessthan3_Moderate.csv',index=False)
# GTM_Mod_data_3.to_csv('GTM_data_CameraTC_grtr_than3_Moderate.csv',index=False)